In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()

api_key = os.environ.get("OPENAI_TOKEN")
client = OpenAI(api_key=api_key)

def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

embedding = get_embedding('hello')
print(embedding)

# df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
# df.to_csv('output/embedded_1k_reviews.csv', index=False)

[0.016751619055867195, -0.055799614638090134, 0.005647437181323767, 0.06618502736091614, 0.008984559215605259, -0.04729178547859192, -0.02814917452633381, 0.06102165952324867, -0.002181964460760355, -0.043741967529058456, 0.009828007780015469, -0.03326853737235069, -0.012226334773004055, -0.028926612809300423, 0.014617327600717545, 0.05826395004987717, -0.07005755603313446, 0.03755179047584534, 0.010092043317854404, 0.04077889770269394, 0.06647840142250061, 0.010121380910277367, -0.005618100054562092, 0.02368989959359169, 0.02175363525748253, 0.011874286457896233, -0.004730645567178726, 0.01930396631360054, 0.024349989369511604, -0.0648941844701767, 0.03326853737235069, -0.0445634126663208, 0.020374780520796776, -0.00495800981298089, 0.010884151794016361, -0.014793351292610168, -0.026271583512425423, 0.03467673063278198, 0.0023341518826782703, -0.03329787775874138, -0.02212035097181797, -0.008544499054551125, 0.05401003733277321, 0.024393994361162186, -7.557802018709481e-05, 0.01559279

In [3]:
def create_and_organize_embeddings(qa_data):
    """Create embeddings and organize for efficient retrieval"""
    
    enhanced_data = []
    
    for qa in qa_data:
        # Create embeddings
        question_emb = get_embedding(qa['question'])
        answer_emb = get_embedding(qa['answer'])
        
        # Add to original structure
        enhanced_qa = {
            **qa,  # Original data
            'question_embedding': question_emb,
            'answer_embedding': answer_emb,
            'embedding_model': 'text-embedding-3-small',
            # 'embedding_created': datetime.now().isoformat()
        }
        
        enhanced_data.append(enhanced_qa)
    
    return enhanced_data

In [5]:
import json 

filename = 'data/qa.json'
with open(filename, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

In [6]:
enhanced_data = create_and_organize_embeddings(qa_data)

In [7]:
enhanced_data

[{'id': '0',
  'question': 'Dobrý den, mohu se zeptat, zda odstín 20 (light beige) make upu značky Kripa (odstín je vyprodaný) je srovnatelný s odstínem light beige nového make upu Naturemma?',
  'answer': 'Odstíny Light beige jsou velmi podobné, je však rozdíl v krytí: 1. KRIPA make-up má vyšší krytí, odstín je tedy více komplexní a může vypadat ve finále světleji 2. Naturemma má lehké až střední krytí, odstín působí na pleti o trochu tmavší. K dispozici jsou vzorky, otestujte to je to nejlepší a pokud by vzorek nevyhovoval, pošlete nenačatý make-up zpět a vyměníme jej. Velmi záleží na způsobu nanášení - houbička, štětec, prsty - různá úroveň krytí.',
  'category': 'makeup',
  'keywords': [],
  'question_embedding': [-0.033931273967027664,
   0.015505618415772915,
   -0.0376662015914917,
   -0.03472353145480156,
   -0.0002606669731903821,
   0.0226585753262043,
   0.025963421911001205,
   0.016965636983513832,
   0.020598703995347023,
   -0.011849913746118546,
   -0.01712408848106861,

In [8]:
with open('data/qa_data_w_vectors.json', 'w', encoding='utf-8') as f:
    json.dump(enhanced_data, f, indent=2, ensure_ascii=False)

In [12]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def load_qa_data_with_embeddings(filename):
    """Load Q&A data with embeddings from JSON file"""
    with open(filename, 'r', encoding='utf-8') as f:
        qa_data = json.load(f)
    return qa_data

def find_similar_questions(query, qa_data_file='data/qa_data_w_vectors.json', top_k=5):
    """
    Find most similar questions to the input query
    
    Args:
        query (str): User's question in Czech
        qa_data_file (str): Path to JSON file with embeddings
        top_k (int): Number of similar questions to return
    
    Returns:
        list: Top k most similar Q&A pairs with similarity scores
    """
    
    # Load data with embeddings
    qa_data = load_qa_data_with_embeddings(qa_data_file)
    
    # Create embedding for user query
    query_embedding = get_embedding(query)
    query_embedding = np.array(query_embedding).reshape(1, -1)
    
    # Extract question embeddings from data
    question_embeddings = []
    for qa in qa_data:
        question_embeddings.append(qa['question_embedding'])
    
    question_embeddings = np.array(question_embeddings)
    
    # Calculate cosine similarities
    similarities = cosine_similarity(query_embedding, question_embeddings)[0]
    
    # Get top k most similar questions
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    # Prepare results
    similar_questions = []
    for idx in top_indices:
        similar_questions.append({
            'id': qa_data[idx]['id'],
            'qa_pair': qa_data[idx],
            'similarity_score': float(similarities[idx]),
            'question': qa_data[idx]['question'],
            'answer': qa_data[idx]['answer']
        })
    
    return similar_questions

In [13]:
# Usage example
similar_questions = find_similar_questions("Jaký make-up má lepší krytí?")

# Display results
for i, result in enumerate(similar_questions, 1):
    print(f"\n{i}. Similarity: {result['similarity_score']:.3f}")
    print(f'ID: {result['id']}')
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer'][:100]}...")


1. Similarity: 0.599
ID: 1
Question: Dobrý den, moc se nevyznám v rozdílu mezi Rozjasňujícim anti age make upem a dvousložkovým make upem. Můžete mi prosím vysvětlit hlavní rozdíl a co lépe kryje nedokonalosti? A proč se už neprodává báze pod make up? Čím tedy nahradit?
Answer: rozdíl mezi make-upem Naturemma a KRIPA. 1. Naturemma má přirozené lehké až střední krytí, lehká kré...

2. Similarity: 0.573
ID: 7
Question: Dobrý den, hledám BB krém nebo možná i jen kompaktní pudr, s nízkou krycí schopností, pro sjednocení pleti. Mám ale problém se trefit do odstínu, abych nevypadala oranžově. Prosím o radu, jak/podle čeho se řídit.
Answer: Dobrý den, v naší nabídce najdete BB krém značky KRIPA - ani jeden odstín nemá oranžový podtón a je ...

3. Similarity: 0.567
ID: 15
Question: Dobrý den, hledám opalovací krém na obličej, který se nebude drolit a loupat a dá se použít pod makeup.
Answer: SPF ochranný krém na obličej, který se nedrolí, drží a lze jej použít, jak jako opalovák tak i jako ..